--- Day 16: Reindeer Maze ---
It's time again for the Reindeer Olympics! This year, the big event is the Reindeer Maze, where the Reindeer compete for the lowest score.

You and The Historians arrive to search for the Chief right as the event is about to start. It wouldn't hurt to watch a little, right?

The Reindeer start on the Start Tile (marked S) facing East and need to reach the End Tile (marked E). They can move forward one tile at a time (increasing their score by 1 point), but never into a wall (#). They can also rotate clockwise or counterclockwise 90 degrees at a time (increasing their score by 1000 points).

To figure out the best place to sit, you start by grabbing a map (your puzzle input) from a nearby kiosk. For example:

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############
There are many paths through this maze, but taking any of the best paths would incur a score of only 7036. This can be achieved by taking a total of 36 steps forward and turning 90 degrees a total of 7 times:


###############
#.......#....E#
#.#.###.#.###^#
#.....#.#...#^#
#.###.#####.#^#
#.#.#.......#^#
#.#.#####.###^#
#..>>>>>>>>v#^#
###^#.#####v#^#
#>>^#.....#v#^#
#^#.#.###.#v#^#
#^....#...#v#^#
#^###.#.#.#v#^#
#S..#.....#>>^#
###############
Here's a second example:

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################
In this maze, the best paths cost 11048 points; following one such path would look like this:

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#^#
#.#.#.#...#...#^#
#.#.#.#.###.#.#^#
#>>v#.#.#.....#^#
#^#v#.#.#.#####^#
#^#v..#.#.#>>>>^#
#^#v#####.#^###.#
#^#v#..>>>>^#...#
#^#v###^#####.###
#^#v#>>^#.....#.#
#^#v#^#####.###.#
#^#v#^........#.#
#^#v#^#########.#
#S#>>^..........#
#################
Note that the path shown above includes one 90 degree turn as the very first move, rotating the Reindeer from facing East to facing North.

Analyze your map carefully. What is the lowest score a Reindeer could possibly get?

To begin, get your puzzle input.

Answer:  
--------------------------------------------  
DFS and keep a current minimum score and replace it if a new one is lower.    
* Assume the maze is all bordered up (with walls '#' all around)  
* The first move, it needs to turn if the East side is a wall.  
* Beside the first move, all turns need to have multiple direction to turn. That means there are more than one directions the current  position can move to, not include the one where it just came from.  
* When a route reaches "dead-end" (no more direction to move to beside the one it came from), the route should be abandoned.  
* If the route has the same node again, doesn't matter in what direction, the route is not a minimal route and should be abandoned.  
* When a route reaches the end goal, calculate the score and compare to the lowest score. If it comes up lower, replace the previous lowest score.  
* To improve the performance,  
    a. if the score is greater than the minimum_score, abandon the route.  
    b. record the lowest score for each node was visited. If a new path gets an equal or higher score, abandon the path.  


In [74]:
##========================================
## DFS using Queue
##========================================
from collections import deque

def is_valid_move(maze, x, y, visited):
    return maze[x][y] != '#' and (x, y) not in visited

def explore_maze(maze, start_x, start_y, start_dir, end_x, end_y):
    paths = []
    start_score = 0
    minimum_score = 99999999999

    ## prepare a same size metrix to record the lowest score for each node (so the subsequent visits will be removed if the scores are higher)
    lstScore = [[minimum_score for _ in range(len(lstMap[0]))] for _ in range(len(lstMap))]
    # lstScore = [[minimum_score]*len(lstMap[0])]*len(lstMap)   # this doesn't work for some reason
    #print(lstScore)

    ## queue structure: [(x, y, dir, score, [path], {visited})]
    queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y)])])
    #queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y, start_dir, start_score)], set([(start_x, start_y)]))])

    count=0
    while queue and count<1000000000:
        count += 1
        x, y, dir, score, path = queue.popleft()
        print("x, y, dir, score, nodeScore, path = ",x, y, dir, score, lstScore[x][y], path)

        # if the node was visited before (by other path) and its score is lower to the previous, update lstScore; or skip the path
        if lstScore[x][y] > score:
            lstScore[x][y] = score
        else:   # stScore[x][y] <= score
            continue

        # If the end node is reached, add the path to paths
        if x == end_x and y == end_y:
            paths.append([(score,count),path])
            minimum_score = score if score<minimum_score else minimum_score
            continue

        # Define moves (right, down, left, up)
        moves = [(0, 1,'>'), (1, 0,'v'), (0, -1,'<'), (-1, 0,'^')]

        # Explore all possible moves
        for move in moves:
            new_x, new_y, new_dir = x + move[0], y + move[1], move[2]
            new_score = score + 1 + (0 if move[2]==dir else 1000)
            if new_score < minimum_score and is_valid_move(maze, new_x, new_y, path):
                new_path = path.copy()
                new_path.append((new_x, new_y))
                queue.append((new_x, new_y, new_dir, new_score, new_path))

    print("node count in queue, number of paths=", count, len(paths))
    for i in range(len(paths)): print(paths[i])
    return minimum_score


##*****************************
# Part I Test sample map
##*****************************
strMap = """###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############"""

## get Map into a list
lstMap = []
for line in strMap.splitlines():
    lstMap.append(line.replace("\n",""))
#print(lstMap)

xbound = len(lstMap)
ybound = len(lstMap[0])

# check default positions first (to save time)
startPoint = (len(lstMap)-2,1) if lstMap[len(lstMap)-2][1] == 'S' else ()
endPoint = (1,len(lstMap[0])-2) if lstMap[1][len(lstMap[0])-2] == 'E' else ()

# if iregular, find them from the map
if startPoint == () and endPoint == ():
    for x in range(len(lstMap)):
        for y in range(len(lstMap[0])):
            if lstMap[x][y]=="S":
                startPoint = (x,y)
            if lstMap[x][y]=="E":
                endPoint = (x, y)
            if startPoint != () and endPoint != ():
                break
        if startPoint != () and endPoint != ():
            break
print("starting position, end position: ", startPoint, endPoint)

## find the starting/current and end/goal positions of the robot
currentX, currentY, currentDir = startPoint[0], startPoint[1], '>'

minimum_score = explore_maze(lstMap, startPoint[0], startPoint[1], currentDir, endPoint[0], endPoint[1])
print("score=", minimum_score)



starting position, end position:  (13, 1) (1, 13)
x, y, dir, score, nodeScore, path =  13 1 > 0 99999999999 [(13, 1)]
x, y, dir, score, nodeScore, path =  13 2 > 1 99999999999 [(13, 1), (13, 2)]
x, y, dir, score, nodeScore, path =  12 1 ^ 1001 99999999999 [(13, 1), (12, 1)]
x, y, dir, score, nodeScore, path =  13 3 > 2 99999999999 [(13, 1), (13, 2), (13, 3)]
x, y, dir, score, nodeScore, path =  11 1 ^ 1002 99999999999 [(13, 1), (12, 1), (11, 1)]
x, y, dir, score, nodeScore, path =  11 2 > 2003 99999999999 [(13, 1), (12, 1), (11, 1), (11, 2)]
x, y, dir, score, nodeScore, path =  10 1 ^ 1003 99999999999 [(13, 1), (12, 1), (11, 1), (10, 1)]
x, y, dir, score, nodeScore, path =  11 3 > 2004 99999999999 [(13, 1), (12, 1), (11, 1), (11, 2), (11, 3)]
x, y, dir, score, nodeScore, path =  9 1 ^ 1004 99999999999 [(13, 1), (12, 1), (11, 1), (10, 1), (9, 1)]
x, y, dir, score, nodeScore, path =  11 4 > 2005 99999999999 [(13, 1), (12, 1), (11, 1), (11, 2), (11, 3), (11, 4)]
x, y, dir, score, nodeScor

In [77]:
##========================================
## DFS using Queue
##========================================
from collections import deque

def is_valid_move(maze, x, y, visited):
    return maze[x][y] != '#' and (x, y) not in visited

def explore_maze(maze, start_x, start_y, start_dir, end_x, end_y):
    paths = []
    start_score = 0
    minimum_score = 99999999999

    ## prepare a same size metrix to record the lowest score for each node (so the subsequent visits will be removed if the scores are higher)
    lstScore = [[minimum_score for _ in range(len(lstMap[0]))] for _ in range(len(lstMap))]
    # lstScore = [[minimum_score]*len(lstMap[0])]*len(lstMap)   # this doesn't work for some reason
    #print(lstScore)

    ## queue structure: [(x, y, dir, score, [path], {visited})]
    queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y)])])
    #queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y, start_dir, start_score)], set([(start_x, start_y)]))])

    count=0
    while queue and count<100000:
        count += 1
        x, y, dir, score, path = queue.popleft()
        #print("x, y, dir, score, nodeScore, path = ",x, y, dir, score, lstScore[x][y], path)

        # if the node was visited before (by other path) and its score is lower to the previous, update lstScore; or skip the path
        if lstScore[x][y] > score:
            lstScore[x][y] = score
        else:   # stScore[x][y] <= score
            continue

        # If the end node is reached, add the path to paths
        if x == end_x and y == end_y:
            paths.append([(score,count),path])
            minimum_score = score if score<minimum_score else minimum_score
            continue

        # Define moves (right, down, left, up)
        moves = [(0, 1,'>'), (1, 0,'v'), (0, -1,'<'), (-1, 0,'^')]

        # Explore all possible moves
        for move in moves:
            new_x, new_y, new_dir = x + move[0], y + move[1], move[2]
            new_score = score + 1 + (0 if move[2]==dir else 1000)
            if new_score < minimum_score and is_valid_move(maze, new_x, new_y, path):
                new_path = path.copy()
                new_path.append((new_x, new_y))
                queue.append((new_x, new_y, new_dir, new_score, new_path))

    print("node count in queue, number of paths=", count, len(paths))
    for i in range(len(paths)): print(paths[i])
    return minimum_score


##***********************************************
## *****   Part I Main Program Start Here   *****
##***********************************************

## get Map into a list
lstMap=[]
with open('D:\Work\AdventOfCode\Data\Day 16 Data.txt','r') as f:
    for line in f:
        lstMap.append(line.replace("\n",""))
#print(lstMap)

xbound = len(lstMap)
ybound = len(lstMap[0])

# check default positions first (to save time)
startPoint = (len(lstMap)-2,1) if lstMap[len(lstMap)-2][1] == 'S' else ()
endPoint = (1,len(lstMap[0])-2) if lstMap[1][len(lstMap[0])-2] == 'E' else ()

# if iregular, find them from the map
if startPoint == () and endPoint == ():
    for x in range(len(lstMap)):
        for y in range(len(lstMap[0])):
            if lstMap[x][y]=="S":
                startPoint = (x,y)
            if lstMap[x][y]=="E":
                endPoint = (x, y)
            if startPoint != () and endPoint != ():
                break
        if startPoint != () and endPoint != ():
            break
print("starting position, end position: ", startPoint, endPoint)

## find the starting/current and end/goal positions of the robot
currentX, currentY, currentDir = startPoint[0], startPoint[1], '>'

minimum_score = explore_maze(lstMap, startPoint[0], startPoint[1], currentDir, endPoint[0], endPoint[1])
print("score=", minimum_score)



starting position, end position:  (139, 1) (1, 139)
node count in queue, number of paths= 40904 7
[(101432, 40010), [(139, 1), (138, 1), (137, 1), (137, 2), (137, 3), (136, 3), (135, 3), (135, 4), (135, 5), (134, 5), (133, 5), (133, 6), (133, 7), (134, 7), (135, 7), (135, 8), (135, 9), (136, 9), (137, 9), (137, 10), (137, 11), (138, 11), (139, 11), (139, 12), (139, 13), (139, 14), (139, 15), (138, 15), (137, 15), (137, 16), (137, 17), (137, 18), (137, 19), (138, 19), (139, 19), (139, 20), (139, 21), (138, 21), (137, 21), (137, 22), (137, 23), (137, 24), (137, 25), (137, 26), (137, 27), (137, 28), (137, 29), (137, 30), (137, 31), (137, 32), (137, 33), (137, 34), (137, 35), (137, 36), (137, 37), (137, 38), (137, 39), (137, 40), (137, 41), (137, 42), (137, 43), (137, 44), (137, 45), (138, 45), (139, 45), (139, 46), (139, 47), (138, 47), (137, 47), (137, 48), (137, 49), (138, 49), (139, 49), (139, 50), (139, 51), (138, 51), (137, 51), (137, 52), (137, 53), (136, 53), (135, 53), (134, 53), 

--- Part Two ---
Now that you know what the best paths look like, you can figure out the best spot to sit.

Every non-wall tile (S, ., or E) is equipped with places to sit along the edges of the tile. While determining which of these tiles would be the best spot to sit depends on a whole bunch of factors (how comfortable the seats are, how far away the bathrooms are, whether there's a pillar blocking your view, etc.), the most important factor is whether the tile is on one of the best paths through the maze. If you sit somewhere else, you'd miss all the action!

So, you'll need to determine which tiles are part of any best path through the maze, including the S and E tiles.

In the first example, there are 45 tiles (marked O) that are part of at least one of the various best paths through the maze:

###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############
In the second example, there are 64 tiles that are part of at least one of the best paths:

#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################
Analyze your map further. How many tiles are part of at least one of the best paths through the maze?

Answer:  
---------------------------------------------  
Need to consider the lowest scores of each node with different direction. When a path visit the node, it might be in different direction, so the score can be 1000 different before it turns. So when we check the score for each node, either consider the direction, or, it seems more efficient, just give 1000 point buffer to let the path passed.



In [95]:
##========================================
## DFS using Queue
##========================================
from collections import deque

def is_valid_move(maze, x, y, visited):
    return maze[x][y] != '#' and (x, y) not in visited

def explore_maze(maze, start_x, start_y, start_dir, end_x, end_y):
    paths = []
    start_score = 0
    minimum_score = 99999999999

    ## prepare a same size metrix to record the lowest score for each node (so the subsequent visits will be removed if the scores are higher)
    lstScore = [[minimum_score for _ in range(len(lstMap[0]))] for _ in range(len(lstMap))]
    # lstScore = [[minimum_score]*len(lstMap[0])]*len(lstMap)   # this doesn't work for some reason
    #print(lstScore)

    ## queue structure: [(x, y, dir, score, [path], {visited})]
    queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y)])])
    #queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y, start_dir, start_score)], set([(start_x, start_y)]))])

    count=0
    while queue and count<1000000000:
        count += 1
        x, y, dir, score, path = queue.popleft()
        print("x, y, dir, score, nodeScore, path = ",x, y, dir, score, lstScore[x][y], path)

        # if the node was visited before (by other path) and its score is lower to the previous, update lstScore; or skip the path
        if lstScore[x][y]+1000 < score: # consider the score can to in different direction (already turned)
            continue
        elif lstScore[x][y] > score:
            lstScore[x][y] = score

        # If the end node is reached, add the path to paths
        if x == end_x and y == end_y:
            paths.append([(score,count),path])
            minimum_score = score if score<minimum_score else minimum_score
            continue

        # Define moves (right, down, left, up)
        moves = [(0, 1,'>'), (1, 0,'v'), (0, -1,'<'), (-1, 0,'^')]

        # Explore all possible moves
        for move in moves:
            new_x, new_y, new_dir = x + move[0], y + move[1], move[2]
            new_score = score + 1 + (0 if move[2]==dir else 1000)
            if new_score < minimum_score and is_valid_move(maze, new_x, new_y, path):
                new_path = path.copy()
                new_path.append((new_x, new_y))
                queue.append((new_x, new_y, new_dir, new_score, new_path))

    print("node count in queue, number of paths, minimum score=", count, len(paths), minimum_score)
    for i in range(len(paths)): print(paths[i])

    # round up best paths
    lstBestPaths = []
    for path in paths:
        if path[0][0]==minimum_score:
            lstBestPaths.append(path[1])
    print("Best Paths:", lstBestPaths)

    # flaten the list and dedup
    lstBestSeats = []
    for row in lstBestPaths:
        lstBestSeats += row
    print("Best Seats:", lstBestSeats)

    return len(set(lstBestSeats))


##*****************************
# Part II Test sample map
##*****************************
strMap = """#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################"""

## get Map into a list
lstMap = []
for line in strMap.splitlines():
    lstMap.append(line.replace("\n",""))
#print(lstMap)

xbound = len(lstMap)
ybound = len(lstMap[0])

# check default positions first (to save time)
startPoint = (len(lstMap)-2,1) if lstMap[len(lstMap)-2][1] == 'S' else ()
endPoint = (1,len(lstMap[0])-2) if lstMap[1][len(lstMap[0])-2] == 'E' else ()

# if iregular, find them from the map
if startPoint == () and endPoint == ():
    for x in range(len(lstMap)):
        for y in range(len(lstMap[0])):
            if lstMap[x][y]=="S":
                startPoint = (x,y)
            if lstMap[x][y]=="E":
                endPoint = (x, y)
            if startPoint != () and endPoint != ():
                break
        if startPoint != () and endPoint != ():
            break
print("starting position, end position: ", startPoint, endPoint)

## find the starting/current and end/goal positions of the robot
currentX, currentY, currentDir = startPoint[0], startPoint[1], '>'

NoOfBestSeats = explore_maze(lstMap, startPoint[0], startPoint[1], currentDir, endPoint[0], endPoint[1])
print("Number of Best Seats=", NoOfBestSeats)




starting position, end position:  (15, 1) (1, 15)
x, y, dir, score, nodeScore, path =  15 1 > 0 99999999999 [(15, 1)]
x, y, dir, score, nodeScore, path =  14 1 ^ 1001 99999999999 [(15, 1), (14, 1)]
x, y, dir, score, nodeScore, path =  13 1 ^ 1002 99999999999 [(15, 1), (14, 1), (13, 1)]
x, y, dir, score, nodeScore, path =  12 1 ^ 1003 99999999999 [(15, 1), (14, 1), (13, 1), (12, 1)]
x, y, dir, score, nodeScore, path =  11 1 ^ 1004 99999999999 [(15, 1), (14, 1), (13, 1), (12, 1), (11, 1)]
x, y, dir, score, nodeScore, path =  10 1 ^ 1005 99999999999 [(15, 1), (14, 1), (13, 1), (12, 1), (11, 1), (10, 1)]
x, y, dir, score, nodeScore, path =  9 1 ^ 1006 99999999999 [(15, 1), (14, 1), (13, 1), (12, 1), (11, 1), (10, 1), (9, 1)]
x, y, dir, score, nodeScore, path =  8 1 ^ 1007 99999999999 [(15, 1), (14, 1), (13, 1), (12, 1), (11, 1), (10, 1), (9, 1), (8, 1)]
x, y, dir, score, nodeScore, path =  7 1 ^ 1008 99999999999 [(15, 1), (14, 1), (13, 1), (12, 1), (11, 1), (10, 1), (9, 1), (8, 1), (7, 1)]

In [96]:
##========================================
## DFS using Queue
##========================================
from collections import deque

def is_valid_move(maze, x, y, visited):
    return maze[x][y] != '#' and (x, y) not in visited

def explore_maze(maze, start_x, start_y, start_dir, end_x, end_y):
    paths = []
    start_score = 0
    minimum_score = 99999999999

    ## prepare a same size metrix to record the lowest score for each node (so the subsequent visits will be removed if the scores are higher)
    lstScore = [[minimum_score for _ in range(len(lstMap[0]))] for _ in range(len(lstMap))]
    # lstScore = [[minimum_score]*len(lstMap[0])]*len(lstMap)   # this doesn't work for some reason
    #print(lstScore)

    ## queue structure: [(x, y, dir, score, [path], {visited})]
    queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y)])])
    #queue = deque([(start_x, start_y, start_dir, start_score, [(start_x, start_y, start_dir, start_score)], set([(start_x, start_y)]))])

    count=0
    while queue and count<1000000000:
        count += 1
        x, y, dir, score, path = queue.popleft()
        #print("x, y, dir, score, nodeScore, path = ",x, y, dir, score, lstScore[x][y], path)

        # if the node was visited before (by other path) and its score is lower to the previous, update lstScore; or skip the path
        if lstScore[x][y]+1000 < score: # consider the score can to in different direction (already turned)
            continue
        elif lstScore[x][y] > score:
            lstScore[x][y] = score

        # If the end node is reached, add the path to paths
        if x == end_x and y == end_y:
            paths.append([(score,count),path])
            minimum_score = score if score<minimum_score else minimum_score
            continue

        # Define moves (right, down, left, up)
        moves = [(0, 1,'>'), (1, 0,'v'), (0, -1,'<'), (-1, 0,'^')]

        # Explore all possible moves
        for move in moves:
            new_x, new_y, new_dir = x + move[0], y + move[1], move[2]
            new_score = score + 1 + (0 if move[2]==dir else 1000)
            if new_score < minimum_score and is_valid_move(maze, new_x, new_y, path):
                new_path = path.copy()
                new_path.append((new_x, new_y))
                queue.append((new_x, new_y, new_dir, new_score, new_path))

    print("node count in queue, number of paths, minimum score=", count, len(paths), minimum_score)
    for i in range(len(paths)): print(paths[i])

    # round up best paths
    lstBestPaths = []
    for path in paths:
        if path[0][0]==minimum_score:
            lstBestPaths.append(path[1])
    print("Best Paths:", lstBestPaths)

    # flaten the list and dedup
    lstBestSeats = []
    for row in lstBestPaths:
        lstBestSeats += row
    print("Best Seats:", lstBestSeats)

    return len(set(lstBestSeats))


##***********************************************
## *****   Part II Main Program Start Here   ****
##***********************************************

## get Map into a list
lstMap=[]
with open('D:\Work\AdventOfCode\Data\Day 16 Data.txt','r') as f:
    for line in f:
        lstMap.append(line.replace("\n",""))
#print(lstMap)

xbound = len(lstMap)
ybound = len(lstMap[0])

# check default positions first (to save time)
startPoint = (len(lstMap)-2,1) if lstMap[len(lstMap)-2][1] == 'S' else ()
endPoint = (1,len(lstMap[0])-2) if lstMap[1][len(lstMap[0])-2] == 'E' else ()

# if iregular, find them from the map
if startPoint == () and endPoint == ():
    for x in range(len(lstMap)):
        for y in range(len(lstMap[0])):
            if lstMap[x][y]=="S":
                startPoint = (x,y)
            if lstMap[x][y]=="E":
                endPoint = (x, y)
            if startPoint != () and endPoint != ():
                break
        if startPoint != () and endPoint != ():
            break
print("starting position, end position: ", startPoint, endPoint)

## find the starting/current and end/goal positions of the robot
currentX, currentY, currentDir = startPoint[0], startPoint[1], '>'

NoOfBestSeats = explore_maze(lstMap, startPoint[0], startPoint[1], currentDir, endPoint[0], endPoint[1])
print("Number of Best Seats=", NoOfBestSeats)




starting position, end position:  (139, 1) (1, 139)
node count in queue, number of paths, minimum score= 128156 32 89460
[(101432, 118851), [(139, 1), (138, 1), (137, 1), (137, 2), (137, 3), (136, 3), (135, 3), (135, 4), (135, 5), (134, 5), (133, 5), (133, 6), (133, 7), (134, 7), (135, 7), (135, 8), (135, 9), (136, 9), (137, 9), (137, 10), (137, 11), (138, 11), (139, 11), (139, 12), (139, 13), (139, 14), (139, 15), (138, 15), (137, 15), (137, 16), (137, 17), (137, 18), (137, 19), (138, 19), (139, 19), (139, 20), (139, 21), (138, 21), (137, 21), (137, 22), (137, 23), (137, 24), (137, 25), (137, 26), (137, 27), (137, 28), (137, 29), (137, 30), (137, 31), (137, 32), (137, 33), (137, 34), (137, 35), (137, 36), (137, 37), (137, 38), (137, 39), (137, 40), (137, 41), (137, 42), (137, 43), (137, 44), (137, 45), (138, 45), (139, 45), (139, 46), (139, 47), (138, 47), (137, 47), (137, 48), (137, 49), (138, 49), (139, 49), (139, 50), (139, 51), (138, 51), (137, 51), (137, 52), (137, 53), (136, 53)